## Final Project: JOUR 561

#### EXPLORING EARTHQUAKE HAZARDS AND RISK IN LOS ANGELES

For this project, I will use multiple kinds of data to visualize the hazards and risks due to earthquakes in LA County. I will use both the aggregation and analysis that we have learnt in class to understand and analyse the data, and I will create maps to visualize this analysis.

I will mostly use the quaternary faults data from LA County, liquefaction zones data, locations of soft-story houses and some race and economic datasets in LA County to conduct my research. 

I will make some of the maps first in QGIS to see if they work, and then create some final ones in Datawrapper.

In [3]:
%load_ext lab_black
import pandas as pd
import geopandas as gpd
import altair as alt

pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

#### Part I: Importing Datasets

**WHAT THIS DATASET IS**: 

**HOW WILL IT BE USED**:


**PRELIMINARY HYPOTHESIS**:


In [4]:
liquefaction = gpd.read_file("../data/geo/Liquefaction_zones.geojson")

**WHAT THIS DATASET IS**: 

**HOW WILL IT BE USED**:


**PRELIMINARY HYPOTHESIS**:

In [5]:
fault_trace = gpd.read_file("../data/geo/Fault_Trace.geojson")

**WHAT THIS DATASET IS**: 

**HOW WILL IT BE USED**:


**PRELIMINARY HYPOTHESIS**:

In [6]:
concrete_buildings = pd.read_csv("../data/raw/la_concrete_buildings.csv")

**WHAT THIS DATASET IS**: 

**HOW WILL IT BE USED**:


**PRELIMINARY HYPOTHESIS**:

In [7]:
ss_apartment = pd.read_csv("../data/raw/soft_story_apartment.csv")

**WHAT THIS DATASET IS**: 

**HOW WILL IT BE USED**:


**PRELIMINARY HYPOTHESIS**:

In [9]:
ss_condo = pd.read_csv("../data/raw/soft_story_condo.csv")

**WHAT THIS DATASET IS**: 

**HOW WILL IT BE USED**:


**PRELIMINARY HYPOTHESIS**:

**WHAT THIS DATASET IS**: 

**HOW WILL IT BE USED**:


**PRELIMINARY HYPOTHESIS**:

#### PART II: Basic Analysis of Datasets

#### PART III: Aggregation and Further Analysis

#### PART IV: Creating Charts with Altair

#### PART V: Preparing dataframes for Mapping